In [2]:
import pandas as pd
import csv
import chardet
from google.colab import files
from IPython.display import display

print("Sube los archivos: ...")
uploaded    = files.upload()
files_list  = list(uploaded.keys())
feed_file   = next(f for f in files_list if 'feed' in f.lower())
export_file = next(f for f in files_list if 'export' in f.lower())

def detect_enc_delim(path):
    raw = open(path, 'rb').read(100_000)
    enc = chardet.detect(raw)['encoding']
    with open(path, 'r', encoding=enc, errors='replace') as f:
        sample = f.read(2048)
        delim  = csv.Sniffer().sniff(sample).delimiter
    return enc, delim

enc_feed, sep_feed = detect_enc_delim(feed_file)
with open(feed_file, 'r', encoding=enc_feed, errors='replace') as f:
    headers = f.readline().strip().split(sep_feed)
cols_feed = [h.strip().strip('"').strip("'") for h in headers]

manufacture_col = next(c for c in cols_feed if 'manufactur' in c.lower())
price_sale_col  = next((c for c in cols_feed
                       if 'price' in c.lower() and 'sale' in c.lower()),
                       next(c for c in cols_feed if 'price' in c.lower()))
quantity_col    = next(c for c in cols_feed if 'quantit' in c.lower())

df_feed = pd.read_csv(
    feed_file,
    sep=sep_feed,
    encoding=enc_feed,
    usecols=[manufacture_col, price_sale_col, quantity_col],
    engine='python',
    on_bad_lines='skip'
)
df_feed[price_sale_col] = pd.to_numeric(df_feed[price_sale_col], errors='coerce')
df_feed[quantity_col]   = pd.to_numeric(df_feed[quantity_col], errors='coerce')
df_feed['stock_value']  = df_feed[price_sale_col] * df_feed[quantity_col]

agg_stock = (
    df_feed
    .groupby(manufacture_col, as_index=False)['stock_value']
    .sum()
    .rename(columns={manufacture_col: 'Marca', 'stock_value': 'Stock Value'})
)

enc_exp, sep_exp = detect_enc_delim(export_file)
with open(export_file, 'r', encoding=enc_exp, errors='replace') as f:
    headers = f.readline().strip().split(sep_exp)
cols_exp = [h.strip().strip('"').strip("'") for h in headers]

brand_col  = next(c for c in cols_exp if 'marca' in c.lower())
sales_col  = next(c for c in cols_exp if 'venta' in c.lower() or 'sales' in c.lower())
margin_col = next(c for c in cols_exp if 'margen' in c.lower() or 'margin' in c.lower())

df_exp = pd.read_csv(
    export_file,
    sep=sep_exp,
    encoding=enc_exp,
    usecols=[brand_col, sales_col, margin_col],
    engine='python',
    on_bad_lines='skip'
)
for col in (sales_col, margin_col):
    df_exp[col] = (
        df_exp[col]
        .astype(str)
        .str.replace(r'\.', '', regex=True)
        .str.replace(r',', '.', regex=True)
    )
    df_exp[col] = pd.to_numeric(df_exp[col], errors='coerce')

agg_sales = (
    df_exp
    .groupby(brand_col, as_index=False)
    .agg({sales_col: 'sum', margin_col: 'sum'})
    .rename(columns={brand_col: 'Marca', sales_col: 'Precio Venta', margin_col: 'Margen'})
)

final_df = pd.merge(agg_sales, agg_stock, on='Marca', how='outer').fillna(0)
final_df['Rotación'] = (final_df['Precio Venta'] / final_df['Stock Value'] * 100).round(2)

sorted_df = final_df.sort_values(by='Stock Value', ascending=False).reset_index(drop=True)
pd.set_option('display.max_rows', None)
display(sorted_df)

sorted_df.to_excel('Rotación-Stock.xlsx', index=False)
files.download('Rotación-Stock.xlsx')

Sube los archivos: ...


Saving feed_1 (34).csv to feed_1 (34).csv
Saving ExportedData - 2025-09-19T082003.460.csv to ExportedData - 2025-09-19T082003.460 (1).csv


,Marca,Precio Venta,Margen,Stock Value,Rotación
0,Asics,66582.073,25029.91,413632.69,16.10
1,Saucony,35365.773,12961.84,291925.73,12.11
2,Hoka,17877.510,5760.24,151889.75,11.77
3,Puma,48396.479,20533.62,137326.02,35.24
4,Brooks,21614.036,8333.05,112196.54,19.26
5,New Balance,20664.443,6970.60,98618.88,20.95
6,Q36.5,4727.020,1359.48,70861.64,6.67
7,ALTRA,13308.241,4007.54,65703.40,20.26
8,Shimano,3823.696,644.06,57775.21,6.62
9,Adidas,4962.720,1695.64,52556.31,9.44


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>